# Batch GIF Processing Notebook

This notebook provides functionality to:
1. Process multiple GIFs from an input folder
2. Resize all GIFs to specified square dimensions
3. Add text overlay with the GIF filename and standard text
4. Add logo overlay at selected positions
5. Compress the resulting GIFs

# Usage
- Place your GIFs in the `input` directory, or keep them at the root of the `api-examples` folder if they were downloaded with `record_gifs_with_wave.ipynb`.
- Run All Cells to process the GIFs and save the results in the `output` directory.
- You can select available fonts for text overlays by running the `font_manager.list_fonts()` command in the 3.2. cell and setting `FONT_NAME` to one of the listed fonts.
- If you wish to use LLM to generate descriptions for the GIFs, install `ollama` and ensure the service is running and the model is pulled.
- Descriptions will be saved in `descriptions.txt` in the current working directory.
- You can customize the logo, text overlays, and GIF quality settings in the respective sections of the notebook.


## 1. Input/Output Settings

In [4]:
# Directory settings
INPUT_DIR = "input"
OUTPUT_DIR = "output"
ASSETS_DIR = "assets"

# GIF Settings
GIF_SIZE = 600  # Size for square output
QUALITY = 30  # GIF quality (1-100)

# Logo Settings
LOGO_FILE = "logo.png"  # Place your logo in files/assets/
LOGO_POSITION = (15, 10)

# Text Overlay Settings
FONT_SIZE = 16
TEXT_COLOR = (255, 255, 255)
STROKE_COLOR = (0, 0, 0)
FONT_NAME = "lucida-grande"  # Use `font_manager.list_fonts()` below in 3.2. to see available fonts

TEXT_2 = "Available in our materials bank"

## 2. Utility Functions

In [5]:
import os
import shutil
import re


def copy_and_clean_gifs(source_folder, target_folder="input"):
    """
    Copy GIF files from source folder to target folder,
    removing numbered duplicates (e.g., removes 'xxx1.gif' if 'xxx.gif' exists)

    Args:
        source_folder (str): Path to source folder containing GIFs
        target_folder (str): Path to target folder (defaults to 'input')
    """

    # Ensure target folder exists
    os.makedirs(target_folder, exist_ok=True)

    # Get all GIF files from source
    gif_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.gif')]

    # Dictionary to store base names and their variations
    file_groups = {}

    # Group files by their base names
    for file in gif_files:
        # Remove .gif extension
        base = file[:-4]
        # Check if the filename ends with a number
        match = re.match(r'(.*?)\d+$', base)

        if match:
            # If it has a number, use the part before the number as key
            key = match.group(1).rstrip()
        else:
            # If no number, use the whole base as key
            key = base

        if key not in file_groups:
            file_groups[key] = []
        file_groups[key].append(file)

    # Copy files, skipping numbered versions if base version exists
    copied_count = 0
    skipped_count = 0

    for base_name, variations in file_groups.items():
        # Sort variations to ensure base version (without number) comes first if it exists
        variations.sort(key=lambda x: (len(x), x))

        # Copy the first variation (usually the base version)
        source_path = os.path.join(source_folder, variations[0])
        target_path = os.path.join(target_folder, variations[0])
        shutil.copy2(source_path, target_path)
        copied_count += 1

        # Count skipped variations
        skipped_count += len(variations) - 1

    print(f"Copied {copied_count} files")
    if skipped_count > 0:
        print(f"Skipped {skipped_count} numbered variations")

# Example usage:
# copy_and_clean_gifs("/path/to/source/folder")
# Or with custom target: copy_and_clean_gifs("/path/to/source", "custom_input")

## 2.1. Copy and Clean GIFs

In [6]:
import json

# GIFs generated by `record_gifs_with_wave.ipynb` are downloaded to the root of `api-examples`
# They need to be copied to input directory and pruned from duplications due to possible bugs.
current_dir = os.getcwd()
print(current_dir)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
print(parent_dir)
copy_and_clean_gifs(parent_dir, INPUT_DIR)

# Some symbols needed to be encoded and decoded
# Load the symbols from the JSON file
with open('symbols_map.json', 'r') as file:
    symbols = json.load(file)

# Extract the "OVER" symbol
SLASH_SYMBOL = symbols["/"]

# Create directories if they don't exist
for directory in [INPUT_DIR, OUTPUT_DIR, ASSETS_DIR]:
    os.makedirs(directory, exist_ok=True)

/Users/mat3ra/code/RED/api-examples/other/media_generation
/Users/mat3ra/code/RED/api-examples
Copied 0 files


## 2.2. List fonts

In [7]:
from other.media_generation.utils.font_manager import FontManager

# Initialize font manager and list available fonts
font_manager = FontManager()
print("Available fonts:")
# print(font_manager.list_fonts())


Available fonts:


## 2.3. Define Text Overlays

In [8]:
def create_text_overlays(filename):
    """Create text overlays using the GIF filename as text_1"""
    # Clean up filename by removing extension and replacing underscores/hyphens with spaces
    clean_name = os.path.splitext(filename)[0].replace(SLASH_SYMBOL, "/")

    return [
        {
            "text": clean_name,
            "position": (10, GIF_SIZE - 10 - FONT_SIZE),  # Bottom left
            "font": FONT_NAME,
            "color": TEXT_COLOR,
            "stroke_width": 2,
            "stroke_fill": STROKE_COLOR
        },
        {
            "text": TEXT_2,
            "position": (GIF_SIZE // 2 + 50, GIF_SIZE - 10 - FONT_SIZE),  # Bottom right
            "font": FONT_NAME,
            "color": TEXT_COLOR,
            "stroke_width": 2,
            "stroke_fill": STROKE_COLOR
        }
    ]


## 3. Process All GIFs

In [9]:
from other.media_generation.utils.gif_processor import GIFProcessor


def process_all_gifs():
    """Process all GIFs in the input directory"""
    # Get logo path
    logo_path = os.path.join(ASSETS_DIR, LOGO_FILE)
    if not os.path.exists(logo_path):
        print(f"Warning: Logo file not found at {logo_path}")
        return

    # Get all GIF files from input directory
    gif_files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith('.gif')]

    if not gif_files:
        print("No GIF files found in input directory")
        return

    print(f"Found {len(gif_files)} GIF files to process")

    # Process each GIF
    for gif_file in gif_files:
        try:
            print(f"\nProcessing: {gif_file}")

            input_path = os.path.join(INPUT_DIR, gif_file)
            output_path = os.path.join(OUTPUT_DIR, f"{gif_file}")

            # Create GIF processor
            gif_processor = GIFProcessor(input_path)

            # Make square and resize
            gif_processor.make_square(size=GIF_SIZE)

            # Add text overlays
            text_overlays = create_text_overlays(gif_file)
            for overlay in text_overlays:
                gif_processor.add_text(
                    text=overlay["text"],
                    position=overlay["position"],
                    font_path=overlay["font"],
                    font_size=FONT_SIZE,
                    color=overlay["color"],
                    stroke_width=overlay["stroke_width"],
                    stroke_fill=overlay["stroke_fill"]
                )

            # Add logo
            gif_processor.add_image_overlay(logo_path, position=LOGO_POSITION)

            # Optimize and save
            gif_processor.optimize(quality=QUALITY)
            gif_processor.save(output_path, optimize=False, quality=QUALITY)

            filename = text_overlays[0]["text"]
            print(f"Filename: {filename}")
            print(f"Successfully processed: {gif_file}")

        except Exception as e:
            print(f"Error processing {gif_file}: {str(e)}")
            continue


# Run the batch processing
process_all_gifs()

Found 3 GIF files to process

Processing: Diamond, N-Substitution.gif
Size on disk: 4.71 MB
Filename: Diamond, N-Substitution
Successfully processed: Diamond, N-Substitution.gif

Processing: Ni, Co-Substitution and Vacancy.gif
Size on disk: 2.55 MB
Filename: Ni, Co-Substitution and Vacancy
Successfully processed: Ni, Co-Substitution and Vacancy.gif

Processing: GaAs, Be-Substitution.gif


KeyboardInterrupt: 